In [46]:
!pip install implicit

In [47]:
from implicit.als import AlternatingLeastSquares
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix
from tqdm import tqdm
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split

In [48]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
FACTORS = 50
REGULARIZATION = 0.01

In [50]:
data_path = "/content/drive/MyDrive/proj_hse/vk_rs"

In [51]:
TRAIN = f"{data_path}/train.csv"
train_df = pd.read_csv(TRAIN)

In [52]:
train_df = train_df.groupby("msno").sample(frac=0.1, random_state=42)

In [53]:
le_msno = LabelEncoder()
le_song_id = LabelEncoder()

le_msno.fit(train_df["msno"])
le_song_id.fit(train_df["song_id"])

train = pd.DataFrame({"msno": le_msno.transform(train_df["msno"].values), "song_id": le_song_id.transform(train_df["song_id"].values), "target": train_df["target"]})

In [95]:
class LightweightClient:
    def __init__(self, client_id: int, user_vector: np.ndarray):
        """
        Args:
            client_id: уникальный ID клиента
            user_vector: вектор факторов только этого пользователя (shape: [factors])
        """
        self.id = client_id
        self.user_vector = user_vector.copy()  # Только свои факторы
        self._interactions = None  # CSR матрица будет инициализирована позже

    def init_interactions(self, item_ids: np.ndarray, targets: np.ndarray, n_items: int):
        """Инициализирует CSR матрицу взаимодействий один раз"""
        self._interactions = csr_matrix(
            (targets, (np.zeros_like(item_ids), item_ids)),
            shape=(1, n_items)
        )

    def local_update(self, global_item_factors: np.ndarray, epsilon: float = None) -> np.ndarray:
        """Возвращает только обновленный вектор пользователя"""
        temp_model = AlternatingLeastSquares(factors=self.user_vector.shape[0],
                                      regularization=REGULARIZATION,
                                      use_gpu=False)
        temp_model.item_factors = global_item_factors
        temp_model.user_factors = np.expand_dims(self.user_vector, axis=0)

        temp_model.partial_fit_users([0], self._interactions)
        updated_vector = temp_model.user_factors[0].copy()

        if epsilon is not None:
            noise = np.random.uniform(low=-epsilon, high=epsilon, size=updated_vector.shape)
            updated_vector += noise

        return updated_vector

# Метрики качества модели, обученной на полных данных

In [54]:
full_data_model = AlternatingLeastSquares(factors=FACTORS,
                                      regularization=REGULARIZATION,
                                      use_gpu=False)

user_item = csr_matrix(
            (train['target'],
            (train['msno'], train['song_id']))
        )

full_data_model.fit(user_item)

train_data, test_data = train_test_split(train, test_size=0.2, random_state=42, shuffle=True)

test_users = test_data["msno"]
test_items = test_data["song_id"]

user_factors = full_data_model.user_factors[test_users]
song_factors = full_data_model.item_factors[test_items]
predicted_scores = np.sum(user_factors * song_factors, axis=1)

auc = roc_auc_score(test_data["target"], predicted_scores)
precision = precision_score(test_data['target'], predicted_scores>0.6)
recall = recall_score(test_data['target'], predicted_scores>0.6)
print("AUC: ", auc)
print("precision: ", precision)
print("recall: ", recall)

  0%|          | 0/15 [00:00<?, ?it/s]

AUC:  0.8009284326995029
precision:  0.9800583657587548
recall:  0.08138454703340199


In [55]:
private_data = train.groupby("msno").sample(frac=0.3, random_state=42)
global_data = train.drop(private_data.index).sort_values(by="msno")

# Дообучение без добавления шума

In [57]:
global_model = AlternatingLeastSquares(factors=FACTORS,
                                      regularization=REGULARIZATION,
                                      use_gpu=False)

user_item = csr_matrix(
            (global_data['target'],
            (global_data['msno'], global_data['song_id']))
        )

global_model.fit(user_item)

  0%|          | 0/15 [00:00<?, ?it/s]

In [58]:
clients = []
for cid in tqdm(private_data["msno"].unique()):
    user_vec = global_model.user_factors[cid].copy()
    client = LightweightClient(cid, user_vec)

    user_data = private_data[private_data["msno"] == cid]
    client.init_interactions(
        item_ids=user_data["song_id"].values,
        targets=user_data["target"].values,
        n_items=global_model.item_factors.shape[0]
    )
    clients.append(client)

100%|██████████| 25371/25371 [00:21<00:00, 1171.63it/s]


In [59]:
updated_vectors = []

for client in tqdm(clients[:int(len(clients)*0.3)]):
    new_vector = client.local_update(global_model.item_factors)
    updated_vectors.append((client.id, new_vector))

for cid, vec in updated_vectors:
    global_model.user_factors[cid] = vec

100%|██████████| 7611/7611 [08:11<00:00, 15.50it/s]


In [60]:
train_data, test_data = train_test_split(global_data, test_size=0.3, random_state=42, shuffle=True)

test_users = test_data["msno"]
test_items = test_data["song_id"]

user_factors = global_model.user_factors[test_users]
song_factors = global_model.item_factors[test_items]
predicted_scores = np.sum(user_factors * song_factors, axis=1)

auc = roc_auc_score(test_data["target"], predicted_scores)
precision = precision_score(test_data['target'], predicted_scores>0.6)
recall = recall_score(test_data['target'], predicted_scores>0.6)
print("AUC: ", auc)
print("precision: ", precision)
print("recall: ", recall)

AUC:  0.7546737056105045
precision:  0.9812558575445174
recall:  0.053805437072819776


# Дообучение с добавлением шума

In [96]:
global_model = AlternatingLeastSquares(factors=FACTORS,
                                      regularization=REGULARIZATION,
                                      use_gpu=False)

user_item = csr_matrix(
            (global_data['target'],
            (global_data['msno'], global_data['song_id']))
        )

global_model.fit(user_item)

  0%|          | 0/15 [00:00<?, ?it/s]

In [97]:
clients = []
for cid in tqdm(private_data["msno"].unique()):
    user_vec = global_model.user_factors[cid].copy()
    client = LightweightClient(cid, user_vec)

    user_data = private_data[private_data["msno"] == cid]
    client.init_interactions(
        item_ids=user_data["song_id"].values,
        targets=user_data["target"].values,
        n_items=global_model.item_factors.shape[0]
    )
    clients.append(client)

100%|██████████| 25371/25371 [00:20<00:00, 1221.78it/s]


In [98]:
updated_vectors = []

for client in tqdm(clients[:int(len(clients)*0.3)]):
    new_vector = client.local_update(global_model.item_factors, epsilon=5*1e-1)
    updated_vectors.append((client.id, new_vector))

for cid, vec in updated_vectors:
    global_model.user_factors[cid] = vec

100%|██████████| 7611/7611 [07:41<00:00, 16.50it/s]


In [99]:
train_data, test_data = train_test_split(global_data, test_size=0.3, random_state=42, shuffle=True)

test_users = test_data["msno"]
test_items = test_data["song_id"]

user_factors = global_model.user_factors[test_users]
song_factors = global_model.item_factors[test_items]
predicted_scores = np.sum(user_factors * song_factors, axis=1)

auc = roc_auc_score(test_data["target"], predicted_scores)
precision = precision_score(test_data['target'], predicted_scores>0.6)
recall = recall_score(test_data['target'], predicted_scores>0.6)
print("AUC: ", auc)
print("precision: ", precision)
print("recall: ", recall)

AUC:  0.6999579969191932
precision:  0.8753121098626716
recall:  0.0720617709029241
